# Lab 1: Tokenization

Splitting a block of text into meaningful subunits is an essential part of processing text. Text could be split into individual characters or words or somewhere in between. A very basic approach is shown below that splits up text using white-space. There's already a shortcoming as the final word 'dog' has punctuation attached to it.

In [1]:
'The quick brown fox jumps over the lazy dog.'.split(' ')

['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog.']

With Transformer models, we do subword tokenizations and split the text up using a prebuilt tokenizer. This has been trained on a large amount of text where it has learned what are common words and which are less common and could be split into parts (that often look like syllables).

First let's load one for a common Transformer model `distilgpt2`. We can load it with the code below. The `distilgpt2` model is a smaller model based upon `gpt2` which is a predecessor to the language model that underpins ChatGPT.

In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

In [31]:
tokenizer.tokenize("I visited Glasgow.")
tokenizer.tokenize("Hello there")

['Hello', 'Ġthere']

 You should four tokens, with some starting with an odd character 'Ġ'. That 'Ġ' denotes that the token starts a new word. Try tokenizing "volcano" below with `tokenizer.tokenize` again. It should be split up into two subword tokens.


In [20]:
tokenizer.tokenize('volcano')

['vol', 'cano']


Along with tokenizing the text into tokens/subtokens, we actually want the tokens to be mapped to numbers. The Transformers take the token indices as input. For example, the token index for the word 'Glasgow' is.

In [21]:
tokenizer.vocab['ĠGlasgow']

23995

`tokenizer.vocab` is a big dictionary mapping subword tokens to their indices. Let's see how big the vocabulary that the `distilgpt2` tokenizer has:

In [22]:
len(tokenizer.vocab)

50257

We could manually map the tokenized output to the token indices. But the tokenizer can do it for us. Pass "I visited Glasgow." into the `tokenizer.encode` function.

In [23]:
tokenizer.encode("I visited Glasgow.")

[40, 8672, 23995, 13]

You can use the `tokenizer.decode` function to convert from a list of indices back to text. Try it out with this list: `[464, 7850, 46922, 4539, 832, 23995, 13]`

In [24]:
tokenizer.decode([464, 7850, 46922, 4539, 832, 23995, 13])


'The river Clyde runs through Glasgow.'

The tokenizer has a lot of parameters to give extra control. For instance, you sometimes need to truncate very long strings (as there is a limit on the length of input to Transformer models). Use the `tokenizer.encode` function to tokenize "Kelvingrove is a beautiful park in Glasgow." and also trim it to only 5 tokens using `truncation=True` and `max_length=5`.

In [25]:
tokenizer.encode("Kelvingrove is a beautiful park in Glasgow.", truncation=True, max_length=5)

[42, 417, 1075, 305, 303]

That should have given you a list of only five token indices.

Now the most common way to use a tokenizer is below which outputs a format ready to pass into a Transformer model. It uses `return_tensors='pt'` which tells it to return PyTorch tensors. PyTorch tensors are a data structure used for deep learning.

The output has the `input_ids` which are the token indices as well as an `attention_mask` which can be used to tell a Transformer to ignore certain tokens. This occurs when using padding to deal with some sequences being shorter than others. That's not the case here, so the attention values are all one.

In [26]:
tokenizer('Kelvingrove is a park in Glasgow.', return_tensors='pt')

ImportError: Unable to convert output to PyTorch tensors format, PyTorch is not installed.

It should be noted that each tokenizer is very specific to the text it was trained on. For instance, below is a tokenizer that was trained on Spanish text.

In [12]:
from transformers import AutoTokenizer

spanish_tokenizer = AutoTokenizer.from_pretrained('datificate/gpt2-small-spanish')

tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

C:\Users\Lenovo\anaconda2024\envs\ml_env_v1\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--datificate--gpt2-small-spanish. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

In [13]:
spanish_tokenizer.tokenize('The river Clyde runs through Glasgow.')

['The', 'Ġri', 'ver', 'ĠClyde', 'Ġr', 'uns', 'Ġth', 'rough', 'ĠGlasgow', '.']

If we give it a previous sentence in English, it tokenizes it very differently and splits up common English words into multiple parts.

In [14]:
spanish_tokenizer.encode('The river Clyde runs through Glasgow.')

[1667, 1316, 778, 44417, 474, 4133, 15848, 26603, 23554, 14]

However, it will tokenize Spanish effectively:

In [15]:
spanish_tokenizer.tokenize('Que te vaya bien')

['Que', 'Ġte', 'Ġvaya', 'Ġbien']

Whereas our `distilgpt2` tokenizer that is trained on English will split up common Spanish words.

In [16]:
tokenizer.tokenize('Que te vaya bien')

['Que', 'Ġte', 'Ġv', 'aya', 'Ġb', 'ien']